# Importing Necessary Libiraries

In [ ]:
import os
import numpy as np 
import pandas as pd
from zipfile import ZipFile
import shutil
import pydub
from os import path
from pydub import AudioSegment
import librosa
import soundfile as sf
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report 
from sklearn import tree
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import pickle

# Slash Character 

In [2]:
character='\\'

# Extracting the required folder

In [23]:
directory=r"C:\Users\HP\Downloads"

os.chdir(directory)

with ZipFile('New DataSet.zip', 'r') as obj:
    obj.extractall()


# Changing into Working Directory


In [4]:
working_dir=r"C:\Users\HP\Downloads\New DataSet"
os.chdir(working_dir)

# Code for extraction of Features


In [5]:
temp=working_dir
temp+=character
files = os.listdir()
file_name=[]
label=[]
feature=[]
count=-1
mfcc_feature=[]
for i in files:
    count+=1
    os.chdir(temp+i)
    for j in os.listdir():
        temp1 , temp2 = librosa.load(j)
        f= librosa.feature.mfcc(y = temp1, sr = temp2)
        feature.append(f)
        mfcc_feature.append(np.mean(f.T,axis=0))
        label.append(count)
        file_name.append(j)

# Code for Flatten and padding of MFCC Features


In [68]:
temp=[]
maximum=0
for i in range(0,len(mfcc_feature)):
    temp.append(np.array(mfcc_feature[i]).flatten())
    if temp[i].shape[0]>maximum:
        maximum=temp[i].shape[0]

padded_values=[]
for i in range(0,len(temp)):
    padded_values.append(np.pad(temp[i],(0,maximum-len(temp[i]))))


# Data Frame

In [ ]:
df=pd.DataFrame()
df["File"]=file_name
df["Labels"]=label
df["MFCC Mean Features"]=padded_values

df=df.drop(columns="File")
df

# Storing into CSV

In [69]:
os.chdir(directory)
df.to_csv('Data.csv', index=False)

# Extracting the Train Data

In [8]:
x_train, x_test, y_train, y_test = train_test_split(df["MFCC Features"],df["Labels"], test_size=0.1, random_state=51)
print('Shape of x_train is ', x_train.shape)
print('Shape of y_train is ', y_train.shape)
print('Shape of x_test is ', x_test.shape)
print('Shape of y_test is ', y_test.shape)
x_train=list(x_train)
x_test=list(x_test)

Shape of x_train is  (5067,)
Shape of y_train is  (5067,)
Shape of x_test is  (563,)
Shape of y_test is  (563,)


# Knn Model

In [9]:
knn = KNeighborsClassifier(n_neighbors=3, weights='uniform',algorithm='auto',leaf_size=30,p=1,metric='minkowski',metric_params=None,n_jobs=None)
knn.fit(x_train, y_train)
prediction= knn.predict(x_test)
knn.score(x_test,y_test)

print ("Accuracy : ", accuracy_score(y_test,prediction)*100)  
print("\t\t\t\t\tReport \n ", classification_report(y_test, prediction))
print("F1 Score : ", f1_score(y_test, prediction, average = 'macro')*100)
print("Precision: ", precision_score(y_test, prediction,average = 'macro')*100)
print("\t\t\tConfusion Matrix  \n", confusion_matrix(y_test,prediction))

Accuracy :  79.21847246891652
					Report 
                precision    recall  f1-score   support

           0       0.57      0.69      0.62        42
           1       0.85      0.93      0.89       337
           2       0.77      0.47      0.58        49
           3       0.72      0.60      0.65       135

    accuracy                           0.79       563
   macro avg       0.73      0.67      0.69       563
weighted avg       0.79      0.79      0.78       563

F1 Score :  68.64614384626745
Precision:  72.50866898309795
			Confusion Matrix  
 [[ 29   9   0   4]
 [  4 313   2  18]
 [  4  12  23  10]
 [ 14  35   5  81]]


# Decision Tree

In [10]:
tr = tree.DecisionTreeClassifier(criterion='gini',splitter='best',max_depth=None,min_samples_split=2,min_samples_leaf=1,min_weight_fraction_leaf=0.0,max_features=None,random_state=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    class_weight=None,
    ccp_alpha=0.0,)
tr = tr.fit(x_train, y_train) 
prediction = tr.predict(x_test)    

print ("Accuracy : ", accuracy_score(y_test,prediction)*100)  
print("\t\t\t\t\tReport \n ", classification_report(y_test, prediction))
print("F1 Score : ", f1_score(y_test, prediction, average = 'macro')*100)
print("Precision: ", precision_score(y_test, prediction,average = 'macro')*100)
print("\t\t\tConfusion Matrix  \n", confusion_matrix(y_test,prediction))

Accuracy :  63.232682060390765
					Report 
                precision    recall  f1-score   support

           0       0.32      0.50      0.39        42
           1       0.79      0.75      0.77       337
           2       0.27      0.29      0.28        49
           3       0.54      0.51      0.53       135

    accuracy                           0.63       563
   macro avg       0.48      0.51      0.49       563
weighted avg       0.65      0.63      0.64       563

F1 Score :  49.09747822669067
Precision:  48.149184018879446
			Confusion Matrix  
 [[ 21  12   2   7]
 [ 24 252  23  38]
 [  4  18  14  13]
 [ 17  37  12  69]]


# Linear SVC

In [11]:
lin_clf = svm.LinearSVC()

lin_clf.fit(x_train, y_train)
prediction = lin_clf.predict(x_test)

print ("Accuracy : ", accuracy_score(y_test,prediction)*100)  
print("\t\t\t\t\tReport \n ", classification_report(y_test, prediction))
print("F1 Score : ", f1_score(y_test, prediction, average = 'macro')*100)
print("Precision: ", precision_score(y_test, prediction,average = 'macro')*100)
print("\t\t\tConfusion Matrix  \n", confusion_matrix(y_test,prediction))

Accuracy :  60.03552397868561
					Report 
                precision    recall  f1-score   support

           0       0.00      0.00      0.00        42
           1       0.60      1.00      0.75       337
           2       0.00      0.00      0.00        49
           3       1.00      0.01      0.01       135

    accuracy                           0.60       563
   macro avg       0.40      0.25      0.19       563
weighted avg       0.60      0.60      0.45       563

F1 Score :  19.110694889746775
Precision:  39.99110320284698
			Confusion Matrix  
 [[  0  42   0   0]
 [  0 337   0   0]
 [  0  49   0   0]
 [  0 134   0   1]]


C:\Users\HP\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in 

# GradientBoostingClassifier

In [12]:
model = GradientBoostingClassifier(n_estimators=600)
model.fit(x_train, y_train)
prediction = model.predict(x_test)

print ("Accuracy : ", accuracy_score(y_test,prediction)*100)  
print("\t\t\t\t\tReport \n ", classification_report(y_test, prediction))
print("F1 Score : ", f1_score(y_test, prediction, average = 'macro')*100)
print("Precision: ", precision_score(y_test, prediction,average = 'macro')*100)
print("\t\t\tConfusion Matrix  \n", confusion_matrix(y_test,prediction))

Accuracy :  74.60035523978685
					Report 
                precision    recall  f1-score   support

           0       0.52      0.31      0.39        42
           1       0.79      0.93      0.85       337
           2       0.69      0.18      0.29        49
           3       0.67      0.64      0.65       135

    accuracy                           0.75       563
   macro avg       0.67      0.51      0.55       563
weighted avg       0.73      0.75      0.72       563

F1 Score :  54.52983693272644
Precision:  66.67132867132867
			Confusion Matrix  
 [[ 13  15   0  14]
 [  5 312   3  17]
 [  2  26   9  12]
 [  5  43   1  86]]


# RandomForestClassifier

In [13]:
forest = RandomForestClassifier(n_estimators=10)
forest = forest.fit(x_train, y_train)
prediction = forest.predict(x_test)

print ("Accuracy : ", accuracy_score(y_test,prediction)*100)  
print("\t\t\t\t\tReport \n ", classification_report(y_test, prediction))
print("F1 Score : ", f1_score(y_test, prediction, average = 'macro')*100)
print("Precision: ", precision_score(y_test, prediction,average = 'macro')*100)
print("\t\t\tConfusion Matrix  \n", confusion_matrix(y_test,prediction))

Accuracy :  73.17939609236235
					Report 
                precision    recall  f1-score   support

           0       0.57      0.38      0.46        42
           1       0.76      0.93      0.84       337
           2       0.53      0.16      0.25        49
           3       0.69      0.55      0.61       135

    accuracy                           0.73       563
   macro avg       0.64      0.51      0.54       563
weighted avg       0.71      0.73      0.70       563

F1 Score :  53.90116096025186
Precision:  63.916031167734
			Confusion Matrix  
 [[ 16  15   2   9]
 [  3 314   4  16]
 [  2  31   8   8]
 [  7  53   1  74]]


# ExtraTreesClassifier

In [14]:
extra_tree = ExtraTreesClassifier( n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
extra_tree=extra_tree.fit(x_train, y_train)
prediction = extra_tree.predict(x_test)

print ("Accuracy : ", accuracy_score(y_test,prediction)*100)  
print("\t\t\t\t\tReport \n ", classification_report(y_test, prediction))
print("F1 Score : ", f1_score(y_test, prediction, average = 'macro')*100)
print("Precision: ", precision_score(y_test, prediction,average = 'macro')*100)
print("\t\t\tConfusion Matrix  \n", confusion_matrix(y_test,prediction))

Accuracy :  75.31083481349911
					Report 
                precision    recall  f1-score   support

           0       0.62      0.36      0.45        42
           1       0.77      0.95      0.85       337
           2       0.72      0.27      0.39        49
           3       0.73      0.57      0.64       135

    accuracy                           0.75       563
   macro avg       0.71      0.53      0.58       563
weighted avg       0.74      0.75      0.73       563

F1 Score :  58.28873447863905
Precision:  71.18456196581197
			Confusion Matrix  
 [[ 15  18   0   9]
 [  4 319   1  13]
 [  3  27  13   6]
 [  2  52   4  77]]


# Storing the pickle files

In [70]:
temp =r"C:\Users\HP\Downloads\PAI Project\Voice Recorder"
os.chdir(temp)

file1="ExtraTree_Model.pkl"
pickle.dump(extra_tree, open(file1,'wb'))

file2="RandomForsest_Model.pkl"
pickle.dump(forest, open(file2,'wb'))

file3="GradientBoosting_Model.pkl"
pickle.dump(model, open(file3,'wb'))

file4="Svm_Model.pkl"
pickle.dump(lin_clf, open(file4,'wb'))

file5="DecisionTree_Model.pkl"
pickle.dump(tr , open(file5,'wb'))

file6="Knn_Model.pkl"
pickle.dump(knn , open(file6,'wb'))